In [0]:
!pip install Unidecode

In [0]:
import pandas as pd
import numpy as np
import glob
import json
import unidecode

In [0]:
# %load_ext autotime

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pre-Processing - Training Data

In [0]:
wiki_pages = glob.glob('/content/drive/My Drive/NLPWikiData/wiki-pages/*')
# wiki_pages = glob.glob('/content/drive/My Drive/NLPWikiData/wiki-pages/wiki-045.jsonl')

In [0]:
df = pd.DataFrame()

In [0]:
train_json_path = "/content/drive/My Drive/NLPWikiData/train.jsonl"
dev_json_path = "/content/drive/My Drive/NLPWikiData/dev.jsonl"
devdf = pd.read_json(train_json_path, lines=True, orient='records')
devdf = devdf[devdf['verifiable'] == 'VERIFIABLE']

In [0]:
devdf

,id,verifiable,label,claim,evidence
2,137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0..."
4,111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1..."
5,89891,VERIFIABLE,REFUTES,Damon Albarn's debut album was released in 2011.,"[[[107201, 120581, Damon_Albarn, 17]]]"
6,181634,VERIFIABLE,SUPPORTS,There is a capital called Mogadishu.,"[[[210946, 218608, Mogadishu, 0]]]"
7,219028,VERIFIABLE,REFUTES,Savages was exclusively a German film.,"[[[260471, 258880, Savages_-LRB-2012_film-RRB-..."
...,...,...,...,...,...
19992,33339,VERIFIABLE,REFUTES,Firefox is the second most popular dog breed.,"[[[49438, 58766, Firefox, 13]]]"
19993,8538,VERIFIABLE,REFUTES,Hermit crabs are arachnids.,"[[[15450, 19262, Hermit_crab, 0], [15450, 1926..."
19994,145641,VERIFIABLE,REFUTES,Michael Hutchence died on a boat.,"[[[168967, 182663, Michael_Hutchence, 15]]]"
19995,87517,VERIFIABLE,SUPPORTS,The Cyclades are located to the southeast of G...,"[[[104709, 118125, Cyclades, 0]]]"


In [0]:
values = devdf['evidence'].explode().explode()
index = values.index

In [0]:
values

2        [289914, 283015, Soul_Food_-LRB-film-RRB-, 0]
2        [291259, 284217, Soul_Food_-LRB-film-RRB-, 0]
2        [293412, 285960, Soul_Food_-LRB-film-RRB-, 0]
2        [337212, 322620, Soul_Food_-LRB-film-RRB-, 0]
2        [337214, 322622, Soul_Food_-LRB-film-RRB-, 0]
                             ...                      
19994          [168967, 182663, Michael_Hutchence, 15]
19995                    [104709, 118125, Cyclades, 0]
19997       [99015, 112132, Trouble_with_the_Curve, 0]
19997       [99015, 112133, Trouble_with_the_Curve, 2]
19997       [99015, 112134, Trouble_with_the_Curve, 5]
Name: evidence, Length: 28625, dtype: object

In [0]:
index

Int64Index([    2,     2,     2,     2,     2,     4,     4,     4,     4,
                4,
            ...
            19993, 19993, 19993, 19993, 19993, 19994, 19995, 19997, 19997,
            19997],
           dtype='int64', length=28625)

In [0]:
train = pd.DataFrame(list(values))


In [0]:
train

,0,1,2,3
0,289914,283015,Soul_Food_-LRB-film-RRB-,0
1,291259,284217,Soul_Food_-LRB-film-RRB-,0
2,293412,285960,Soul_Food_-LRB-film-RRB-,0
3,337212,322620,Soul_Food_-LRB-film-RRB-,0
4,337214,322622,Soul_Food_-LRB-film-RRB-,0
...,...,...,...,...
28620,168967,182663,Michael_Hutchence,15
28621,104709,118125,Cyclades,0
28622,99015,112132,Trouble_with_the_Curve,0
28623,99015,112133,Trouble_with_the_Curve,2


In [0]:
train['claim'] = devdf['claim'][index].reset_index()['claim']
train['id'] = devdf['id'][index].reset_index()['id']
train['label'] = devdf['label'][index].reset_index()['label']
del train[0]
del train[1]
train.columns = ['id', 'sentence', 'claim', 'claim_id', 'label']

In [0]:
train

,id,sentence,claim,claim_id,label
0,Soul_Food_-LRB-film-RRB-,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS
1,Soul_Food_-LRB-film-RRB-,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS
2,Soul_Food_-LRB-film-RRB-,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS
3,Soul_Food_-LRB-film-RRB-,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS
4,Soul_Food_-LRB-film-RRB-,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS
...,...,...,...,...,...
28620,Michael_Hutchence,15,Michael Hutchence died on a boat.,145641,REFUTES
28621,Cyclades,0,The Cyclades are located to the southeast of G...,87517,SUPPORTS
28622,Trouble_with_the_Curve,0,Trouble with the Curve is a television show.,81957,REFUTES
28623,Trouble_with_the_Curve,2,Trouble with the Curve is a television show.,81957,REFUTES


In [0]:
train['match_id'] = train['id'].apply(lambda x: unidecode.unidecode(x))

In [0]:
# train.iloc[219683]

id                            Björn_Ulvaeus
sentence                                   0
claim       Tim Rice worked with a composer.
claim_id                              208718
label                               SUPPORTS
match_id                       Bjorn_Ulvaeus
Name: 219683, dtype: object

In [0]:
# def split_into_id_and_lines(s):
#     lines = s.split('\n')
#     result = []
#     for line in lines:
#         sentence_id, text = line.split('\t', 1)
#         result.append([sentence_id, text])
#     return result

def merge_dfs(pages, wiki_dump, train, i):
    # tmp = wiki_dump[wiki_dump['id'].isin(pages)]
    # # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    # train = train.merge(tmp, how='left', left_on='id', right_on='id')
    # if i > 0:
    #     train['text'] = train['text_x'].fillna(train['text_y'])
    #     train['lines'] = train['lines_x'].fillna(train['lines_y'])
    #     train = train.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y'], axis=1)
    # return train


    tmp = wiki_dump[wiki_dump['match_id'].isin(pages)]
    # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    train = train.merge(tmp, how='left', left_on='match_id', right_on='match_id')
    if i > 0:
        train['id'] = train['id_x'].fillna(train['id_y'])
        train['text'] = train['text_x'].fillna(train['text_y'])
        train['lines'] = train['lines_x'].fillna(train['lines_y'])
        train = train.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y', 'id_x', 'id_y'], axis=1)
    return train

In [0]:
batch_size = 10
wiki_pages_chunks = np.array_split(wiki_pages, batch_size)
i = 0
for chunks in wiki_pages_chunks:
    df = pd.DataFrame()
    for f in chunks:
        d = pd.read_json(f, orient='records', lines=True)
        df = df.append(d, ignore_index=True)
        print(f'read {f}')
    del d
    # train = merge_dfs(train['id'], df, train, i)
    df['match_id'] = df['id'].apply(lambda x: unidecode.unidecode(x))
    train = merge_dfs(train['match_id'], df, train, i)
    
    print(f'done chunk {i}')
    i += 1
    # if i ==2:
      # break
    

read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-001.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-002.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-003.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-004.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-005.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-006.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-007.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-008.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-009.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-010.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-011.jsonl
done chunk 0
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-012.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-013.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-014.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pag

In [0]:
train.drop(labels=['text'], axis=1, inplace=True)

In [0]:
train

,sentence,claim,claim_id,label,match_id,id,lines
0,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
1,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
2,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
3,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
4,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
...,...,...,...,...,...,...,...
28644,15,Michael Hutchence died on a boat.,145641,REFUTES,Michael_Hutchence,Michael_Hutchence,0\tMichael Kelland John Hutchence -LRB- 22 Jan...
28645,0,The Cyclades are located to the southeast of G...,87517,SUPPORTS,Cyclades,Cyclades,0\tThe Cyclades -LRB- -LSB- ˈsɪklədiːz -RSB- Κ...
28646,0,Trouble with the Curve is a television show.,81957,REFUTES,Trouble_with_the_Curve,Trouble_with_the_Curve,0\tTrouble with the Curve is a 2012 sports-dra...
28647,2,Trouble with the Curve is a television show.,81957,REFUTES,Trouble_with_the_Curve,Trouble_with_the_Curve,0\tTrouble with the Curve is a 2012 sports-dra...


In [0]:
path = "/content/drive/My Drive/NLPWikiData/raw2_dev_data.json"
train.to_json(path, orient='index')

In [0]:
raw_data = pd.read_json("/content/drive/My Drive/NLPWikiData/raw2_dev_data.json",orient='index')

In [0]:
raw_data.head(2)

,sentence,claim,claim_id,label,match_id,id,lines
0,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...
1,0,Fox 2000 Pictures released the film Soul Food.,137334,SUPPORTS,Soul_Food_-LRB-film-RRB-,Soul_Food_-LRB-film-RRB-,0\tSoul Food is a 1997 American comedy-drama f...


In [0]:
raw_data.shape

(28649, 7)

In [0]:
def rule_split(lines_b):
    '''
    Important function!
    This function clean the lines in original preprocessed wiki Pages.
    :param lines_b:
    :return: a list of sentence (responded to each sentence)
    '''
    lines = []

    i = 0
    while True:
        start = f'\n{i}\t'
        end = f'\n{i + 1}\t'
        start_index = lines_b.find(start)
        end_index = lines_b.find(end)

        if end_index == -1:
            extra = f'\n{i + 2}\t'
            extra_1 = f'\n{i + 3}\t'
            extra_2 = f'\n{i + 4}\t'

            # print(lines_b[start_index:])
            lines.append(lines_b[start_index:])
            # print('What?')
            # print(lines_b)
            # print(extra, extra_1)
            # print(lines_b.find(extra))
            # print(lines_b.find(extra_1))
            # print(not (lines_b.find(extra) == -1 and lines_b.find(extra_1) == -1))

            if not (lines_b.find(extra) == -1
                    and lines_b.find(extra_1) == -1
                    and lines_b.find(extra_2) == -1):
                print(lines_b)
                print(extra, extra_1)
                print('Error')
            break

        lines.append(lines_b[start_index:end_index])
        i += 1

    return lines

In [0]:
def lines_to_items(lines):
    lines_list = []

    for i, line in enumerate(lines):
        line_item = dict()

        line_item_list = line.split('\t')

        line_num = line_item_list[0]
        # print(line_num)
        if not line_num.isdigit():
            print("None digit")
            print(lines)
        else:
            line_num = int(line_num)

        if int(line_num) != i:
            print("Line num mismath")
            print(int(line_num), i)

        line_item['line_num'] = line_num
        line_item['sentence'] = []
        line_item['h_links'] = []

        if len(line_item_list) <= 1:
            lines_list.append(line_item)
            continue

        sent = line_item_list[1].strip()
        h_links = line_item_list[2:]

        if 'thumb' in h_links:
            h_links = []
        else:
            h_links = list(filter(lambda x: len(x) > 0, h_links))

        line_item['sentence'] = sent
        line_item['h_links'] = h_links
        # print(line_num, sent)
        # print(len(h_links))
        # print(sent)
        # assert sent[-1] == '.'

        if len(h_links) % 2 != 0:
            
            for w in lines:
                print(w)
            print("Term mod 2 != 0")

            print("List:", line_item_list)
            print(line_num, sent)
            print(h_links)
            print()

        lines_list.append(line_item)

    # print(len(lines_list), lines_list[-1]['line_num'] + 1)
    # assert len(lines_list) == int(lines_list[-1]['line_num'] + 1)
    # string_line_item = json.dumps(lines_list)
    return lines_list

In [0]:
from tqdm import tqdm

processed_data = []
for data in tqdm(raw_data.itertuples()):
  # print(data)
  try:
    lines_with_slash_n = '\n'+data[7]
  except:
    print("Data:  ", data)
    continue

  lines = rule_split(lines_with_slash_n)
  lines = [line.strip().replace('\n', '') for line in lines]
  
  if len(lines) == 1 and lines[0] == '':
      lines = ['0']
  all_sentences = lines_to_items(lines)

  for i,sentence in enumerate(all_sentences):
    # temp_dict['doc_name'] = data[1]
    if len(sentence['sentence']) <3:
      continue
    temp_dict = dict()
    temp_dict['claim_id'] = data[3]
    temp_dict['claim'] = data[2]
    temp_dict['sentence'] = sentence['sentence']
    temp_dict['label'] = data[4]
    temp_dict['sent_label'] = sentence['line_num'] == data[1]
    # only for dev data
    temp_dict['sentno'] = sentence['line_num']
    temp_dict['doc_id'] = data[6]

    processed_data.append(temp_dict)


28649it [00:03, 7993.48it/s]


In [0]:
data

Pandas(Index=264961, sentence=8, claim='Susan Sarandon is an award winner.', claim_id=152180, label='SUPPORTS', match_id='Susan_Sarandon', id='Susan_Sarandon', lines="0\tSusan Abigail Sarandon -LRB- -LSB- səˈrændən -RSB- née Tomalin ; born October 4 , 1946 -RRB- is an American actress and activist .\n1\tShe is an Academy Award and BAFTA Award winner who is also known for her social and political activism for a variety of causes .\tAcademy Award\tAcademy Award\tBAFTA Award\tBAFTA Film Award\n2\tShe was appointed a UNICEF Goodwill Ambassador in 1999 and received the Action Against Hunger Humanitarian Award in 2006 .\tUNICEF Goodwill Ambassador\tUNICEF Goodwill Ambassador\tAction Against Hunger\tAction Against Hunger\n3\t\n4\t\n5\tSarandon began her career in the 1970 film Joe , before appearing in the soap opera A World Apart -LRB- 1970 -- 71 -RRB- .\tJoe\tJoe (1970 film)\tA World Apart\tA World Apart (TV series)\n6\tIn 1975 , she starred in the cult classic film The Rocky Horror Picture

In [0]:
lines[1]

'1\tShe is an Academy Award and BAFTA Award winner who is also known for her social and political activism for a variety of causes .\tAcademy Award\tAcademy Award\tBAFTA Award\tBAFTA Film Award'

In [0]:
all_sentences

[{'h_links': [],
  'line_num': 0,
  'sentence': 'Susan Abigail Sarandon -LRB- -LSB- səˈrændən -RSB- née Tomalin ; born October 4 , 1946 -RRB- is an American actress and activist .'},
 {'h_links': ['Academy Award',
   'Academy Award',
   'BAFTA Award',
   'BAFTA Film Award'],
  'line_num': 1,
  'sentence': 'She is an Academy Award and BAFTA Award winner who is also known for her social and political activism for a variety of causes .'},
 {'h_links': ['UNICEF Goodwill Ambassador',
   'UNICEF Goodwill Ambassador',
   'Action Against Hunger',
   'Action Against Hunger'],
  'line_num': 2,
  'sentence': 'She was appointed a UNICEF Goodwill Ambassador in 1999 and received the Action Against Hunger Humanitarian Award in 2006 .'},
 {'h_links': [], 'line_num': 3, 'sentence': []},
 {'h_links': [], 'line_num': 4, 'sentence': []},
 {'h_links': ['Joe',
   'Joe (1970 film)',
   'A World Apart',
   'A World Apart (TV series)'],
  'line_num': 5,
  'sentence': 'Sarandon began her career in the 1970 film

In [0]:
processed_data

[{'claim': 'Fox 2000 Pictures released the film Soul Food.',
  'claim_id': 137334,
  'doc_id': 'Soul_Food_-LRB-film-RRB-',
  'label': 'SUPPORTS',
  'sent_label': True,
  'sentence': "Soul Food is a 1997 American comedy-drama film produced by Kenneth `` Babyface '' Edmonds , Tracey Edmonds and Robert Teitel and released by Fox 2000 Pictures .",
  'sentno': 0},
 {'claim': 'Fox 2000 Pictures released the film Soul Food.',
  'claim_id': 137334,
  'doc_id': 'Soul_Food_-LRB-film-RRB-',
  'label': 'SUPPORTS',
  'sent_label': False,
  'sentence': 'Featuring an ensemble cast , the film stars Vanessa L. Williams , Vivica A. Fox , Nia Long , Michael Beach , Mekhi Phifer , Jeffrey D. Sams , Irma P. Hall , Gina Ravera and Brandon Hammond .',
  'sentno': 1},
 {'claim': 'Fox 2000 Pictures released the film Soul Food.',
  'claim_id': 137334,
  'doc_id': 'Soul_Food_-LRB-film-RRB-',
  'label': 'SUPPORTS',
  'sent_label': False,
  'sentence': 'Written and directed by George Tillman , Jr. -- in his major 

In [0]:
new_df = pd.DataFrame(processed_data)
# only for dev data
new_df['org_sentence'] = new_df['sentence']

In [0]:
new_df.keys()

Index(['claim_id', 'claim', 'sentence', 'label', 'sent_label', 'sentno',
       'doc_id', 'org_sentence'],
      dtype='object')

In [0]:
# new_df.claim_id.astype(str).str

In [0]:
# only for dev data
seperator = "{#--#}"
new_df['docid_claimid_sentno'] = new_df.doc_id + seperator + new_df.claim_id.astype(str) + seperator + new_df.sentno.astype(str)
new_df.drop(labels=['doc_id', 'sentno'], axis=1, inplace=True)
new_df = new_df.drop_duplicates().reset_index(drop=True)

In [0]:
new_df.to_csv("/content/drive/My Drive/NLPWikiData/processed2_dev_data.csv",index=False)

In [0]:
done_data = pd.read_csv("/content/drive/My Drive/NLPWikiData/processed2_dev_data.csv")
# training_data.groupby('claim_id').sum()

In [0]:
done_data.sent_label.value_counts()

False    144632
True      22265
Name: sent_label, dtype: int64

In [0]:
done_data.head(8)

,claim_id,claim,sentence,label,sent_label,org_sentence,docid_claimid_sentno
0,137334,Fox 2000 Pictures released the film Soul Food.,Soul Food is a 1997 American comedy-drama film...,SUPPORTS,True,Soul Food is a 1997 American comedy-drama film...,Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}0
1,137334,Fox 2000 Pictures released the film Soul Food.,"Featuring an ensemble cast , the film stars Va...",SUPPORTS,False,"Featuring an ensemble cast , the film stars Va...",Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}1
2,137334,Fox 2000 Pictures released the film Soul Food.,"Written and directed by George Tillman , Jr. -...",SUPPORTS,False,"Written and directed by George Tillman , Jr. -...",Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}2
3,137334,Fox 2000 Pictures released the film Soul Food.,Tillman based the family in the film on his ow...,SUPPORTS,False,Tillman based the family in the film on his ow...,Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}5
4,137334,Fox 2000 Pictures released the film Soul Food.,"In 2000 , Showtime premiered a one-hour televi...",SUPPORTS,False,"In 2000 , Showtime premiered a one-hour televi...",Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}6
5,137334,Fox 2000 Pictures released the film Soul Food.,"In 2015 , it was announced that 20th Century F...",SUPPORTS,False,"In 2015 , it was announced that 20th Century F...",Soul_Food_-LRB-film-RRB-{#--#}137334{#--#}7
6,111897,Telemundo is a English-language television net...,Telemundo -LRB- -LSB- teleˈmundo -RSB- -RRB- i...,REFUTES,True,Telemundo -LRB- -LSB- teleˈmundo -RSB- -RRB- i...,Telemundo{#--#}111897{#--#}0
7,111897,Telemundo is a English-language television net...,It is the second largest provider of Spanish c...,REFUTES,False,It is the second largest provider of Spanish c...,Telemundo{#--#}111897{#--#}1


In [0]:
import pandas as pd

file_data_path="/content/drive/My Drive/NLPWikiData/sen_preds_output2.jsonl"
temp_df = pd.read_json(file_data_path, lines=True, orient='records')
temp2_df = pd.read_csv("/content/drive/My Drive/NLPWikiData/fever_data_output.csv")


In [0]:
temp_df

,id,claim,sentences,page_ids,indices
0,137334,Fox 2000 Pictures released the film Soul Food.,[Soul Food is a 1997 American comedy-drama fil...,"[Soul_Food_-LRB-film-RRB-, Soul_Food_-LRB-film...","[0, 1, 2, 5, 7]"
1,111897,Telemundo is a English-language television net...,[Hispanic Americans and Latino Americans -LRB-...,"[Hispanic_and_Latino_Americans, Telemundo, Tel...","[0, 0, 0, 3, 16]"
2,89891,Damon Albarn's debut album was released in 2011.,"[Damon Albarn , OBE -LRB- -LSB- ˈdeɪmən_ˈælbɑr...","[Damon_Albarn, Damon_Albarn, Damon_Albarn, Dam...","[0, 1, 16, 17, 12]"
3,181634,There is a capital called Mogadishu.,[Villa Somalia is the official residential pal...,"[Mogadishu, Mogadishu, Mogadishu, Mogadishu, M...","[20, 32, 0, 27, 25]"
4,219028,Savages was exclusively a German film.,"[The film was released on July 6 , 2012 , and ...","[Savages_-LRB-2012_film-RRB-, Savages_-LRB-201...","[6, 3, 0, 4, 5]"
...,...,...,...,...,...
13327,33339,Firefox is the second most popular dog breed.,[Mozilla Firefox -LRB- or simply Firefox -RRB-...,"[Firefox, Firefox, Firefox, Firefox, Firefox]","[0, 13, 14, 7, 8]"
13328,8538,Hermit crabs are arachnids.,[The Decapoda or decapods -LRB- literally `` t...,"[Decapoda, Arachnid, Decapoda, Hermit_crab, He...","[0, 0, 4, 0, 3]"
13329,145641,Michael Hutchence died on a boat.,[Michael Kelland John Hutchence -LRB- 22 Janua...,"[Michael_Hutchence, Michael_Hutchence, Michael...","[0, 3, 15, 1, 4]"
13330,87517,The Cyclades are located to the southeast of G...,[The Cyclades -LRB- -LSB- ˈsɪklədiːz -RSB- Κυκ...,"[Cyclades, Cyclades, Cyclades, Cyclades]","[0, 3, 2, 1]"


In [0]:
temp2_df

,Unnamed: 0,label,predicted_label,predicted_evidence,evidence
0,0,SUPPORTS,SUPPORTS,"[['Soul_Food_-LRB-film-RRB-', 0], ['Soul_Food_...","[[[289914, 283015, 'Soul_Food_-LRB-film-RRB-',..."
1,1,REFUTES,REFUTES,"[['Hispanic_and_Latino_Americans', 0], ['Telem...","[[[131371, 146144, 'Telemundo', 0]], [[131371,..."
2,2,REFUTES,REFUTES,"[['Damon_Albarn', 0], ['Damon_Albarn', 1], ['D...","[[[107201, 120581, 'Damon_Albarn', 17]]]"
3,3,SUPPORTS,SUPPORTS,"[['Mogadishu', 20], ['Mogadishu', 32], ['Mogad...","[[[210946, 218608, 'Mogadishu', 0]]]"
4,4,REFUTES,REFUTES,"[['Savages_-LRB-2012_film-RRB-', 6], ['Savages...","[[[260471, 258880, 'Savages_-LRB-2012_film-RRB..."
...,...,...,...,...,...
13327,13327,REFUTES,REFUTES,"[['Firefox', 0], ['Firefox', 13], ['Firefox', ...","[[[49438, 58766, 'Firefox', 13]]]"
13328,13328,REFUTES,REFUTES,"[['Decapoda', 0], ['Arachnid', 0], ['Decapoda'...","[[[15450, 19262, 'Hermit_crab', 0], [15450, 19..."
13329,13329,REFUTES,REFUTES,"[['Michael_Hutchence', 0], ['Michael_Hutchence...","[[[168967, 182663, 'Michael_Hutchence', 15]]]"
13330,13330,SUPPORTS,SUPPORTS,"[['Cyclades', 0], ['Cyclades', 3], ['Cyclades'...","[[[104709, 118125, 'Cyclades', 0]]]"


In [0]:
temp2_df.evidence

0    [[[177592, 190354, 'Republic_of_Macedonia', 1]...
Name: evidence, dtype: object

# Pre-Processing Test Data

In [0]:
wiki_pages = glob.glob('/content/drive/My Drive/NLPWikiData/wiki-pages/*')
test_json_path = "/content/drive/My Drive/NLPWikiData/doc_ret_test3.jsonl"

In [0]:
test_df = pd.DataFrame()

In [0]:
testdf = pd.read_json(test_json_path, lines=True, orient='records')
# testdf = testdf[testdf['verifiable'] == 'VERIFIABLE']

In [0]:
testdf

,id,claim,pred_evidence
0,89296,Henry Spencer is played by a Greek actor.,"[Henry_Spencer_-LRB-disambiguation-RRB-, Henry..."
1,78554,John Ritter died in October.,"[October_-LRB-Whitacre-RRB-, October_-LRB-film..."
2,83809,13 Reasons Why is the only television series o...,"[Maneater_-LRB-film_series-RRB-, Ludwig_II_-LR..."
3,49758,Playboy is a magazine.,"[Playboy_-LRB-lifestyle-RRB-, Playboy_-LRB-The..."
4,22973,Alternative metal is the genre in which Alice ...,"[Alice_-LRB-programming_language-RRB-, Alice_-..."
...,...,...,...
19993,35276,Ingolfr Arnarson sailed for dead bodies.,"[Inna_Trazhukova, Hippodamia_-LRB-horse-RRB-, ..."
19994,69842,Alaska shares a border with the west side of B...,"[Alaska_-LRB-Maggie_Rogers_song-RRB-, Alaska_-..."
19995,6710,Amud 1 has the largest cranial capacity of any...,[Amud]
19996,93269,Aamir Khan has been the recipient of three Bes...,"[Aamir_Khan, Mainichi_Film_Award_for_Best_Acto..."


In [0]:
testdf.dtypes

id                int64
claim            object
pred_evidence    object
dtype: object

In [0]:
values = testdf['pred_evidence'].explode()
index = values.index

In [0]:
values

0              Henry_Spencer_-LRB-disambiguation-RRB-
0                       Henry_Spencer_-LRB-Psych-RRB-
0                                       Henry_Spencer
0                    Henry_Spencer_-LRB-fl._1402-RRB-
0                                 Lowndes_Henry_Davis
                             ...                     
19997    The_Handmaid's_Tale_-LRB-disambiguation-RRB-
19997                             The_Handmaid's_Tale
19997              The_Handmaid's_Tale_-LRB-film-RRB-
19997         The_Handmaid's_Tale_-LRB-TV_series-RRB-
19997             The_Handmaid's_Tale_-LRB-opera-RRB-
Name: pred_evidence, Length: 174496, dtype: object

In [0]:
index

Int64Index([    0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,
            ...
            19996, 19996, 19996, 19996, 19996, 19997, 19997, 19997, 19997,
            19997],
           dtype='int64', length=174496)

In [0]:
test = pd.DataFrame(list(values))

In [0]:
test

,0
0,Henry_Spencer_-LRB-disambiguation-RRB-
1,Henry_Spencer_-LRB-Psych-RRB-
2,Henry_Spencer
3,Henry_Spencer_-LRB-fl._1402-RRB-
4,Lowndes_Henry_Davis
...,...
174491,The_Handmaid's_Tale_-LRB-disambiguation-RRB-
174492,The_Handmaid's_Tale
174493,The_Handmaid's_Tale_-LRB-film-RRB-
174494,The_Handmaid's_Tale_-LRB-TV_series-RRB-


In [0]:
test['claim']=testdf['claim'][index].reset_index()['claim']

In [0]:
test['claim_id']=testdf['id'][index].reset_index()['id']

In [0]:
test

,0,claim,claim_id
0,Henry_Spencer_-LRB-disambiguation-RRB-,Henry Spencer is played by a Greek actor.,89296
1,Henry_Spencer_-LRB-Psych-RRB-,Henry Spencer is played by a Greek actor.,89296
2,Henry_Spencer,Henry Spencer is played by a Greek actor.,89296
3,Henry_Spencer_-LRB-fl._1402-RRB-,Henry Spencer is played by a Greek actor.,89296
4,Lowndes_Henry_Davis,Henry Spencer is played by a Greek actor.,89296
...,...,...,...
174491,The_Handmaid's_Tale_-LRB-disambiguation-RRB-,The Handmaid's Tale was approved for a second ...,51709
174492,The_Handmaid's_Tale,The Handmaid's Tale was approved for a second ...,51709
174493,The_Handmaid's_Tale_-LRB-film-RRB-,The Handmaid's Tale was approved for a second ...,51709
174494,The_Handmaid's_Tale_-LRB-TV_series-RRB-,The Handmaid's Tale was approved for a second ...,51709


In [0]:
test['claim'] = testdf['claim'][index].reset_index()['claim']
test['claim_id'] = testdf['id'][index].reset_index()['id']
# test['id'] = testdf['pred_evidence'][index].reset_index()['pred_evidence']
# test['label'] = testdf['label'][index].reset_index()['label']
# del test[0]
# del test[1]
test.columns = ['id', 'claim', 'claim_id']

In [0]:
# test.id.dropna(inplace=True)

In [0]:
test.dtypes

id          object
claim       object
claim_id     int64
dtype: object

In [0]:
def a(x):
  try:
    return unidecode.unidecode(x)
    
  except Exception as e:
    print(e, x)
    raise Exception


In [0]:
test['match_id'] = test['id'].apply(lambda x: unidecode.unidecode(x))

In [0]:
test

,id,claim,claim_id,match_id
0,Henry_Spencer_-LRB-disambiguation-RRB-,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-disambiguation-RRB-
1,Henry_Spencer_-LRB-Psych-RRB-,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-Psych-RRB-
2,Henry_Spencer,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer
3,Henry_Spencer_-LRB-fl._1402-RRB-,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-fl._1402-RRB-
4,Lowndes_Henry_Davis,Henry Spencer is played by a Greek actor.,89296,Lowndes_Henry_Davis
...,...,...,...,...
174491,The_Handmaid's_Tale_-LRB-disambiguation-RRB-,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-disambiguation-RRB-
174492,The_Handmaid's_Tale,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale
174493,The_Handmaid's_Tale_-LRB-film-RRB-,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-film-RRB-
174494,The_Handmaid's_Tale_-LRB-TV_series-RRB-,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-TV_series-RRB-


In [0]:
# def split_into_id_and_lines(s):
#     lines = s.split('\n')
#     result = []
#     for line in lines:
#         sentence_id, text = line.split('\t', 1)
#         result.append([sentence_id, text])
#     return result

def merge_dfs(pages, wiki_dump, test, i):
    # tmp = wiki_dump[wiki_dump['id'].isin(pages)]
    # # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    # train = train.merge(tmp, how='left', left_on='id', right_on='id')
    # if i > 0:
    #     train['text'] = train['text_x'].fillna(train['text_y'])
    #     train['lines'] = train['lines_x'].fillna(train['lines_y'])
    #     train = train.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y'], axis=1)
    # return train


    tmp = wiki_dump[wiki_dump['match_id'].isin(pages)]
    # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    test = test.merge(tmp, how='left', left_on='match_id', right_on='match_id')
    if i > 0:
        test['id'] = test['id_x'].fillna(test['id_y'])
        test['text'] = test['text_x'].fillna(test['text_y'])
        test['lines'] = test['lines_x'].fillna(test['lines_y'])
        test = test.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y', 'id_x', 'id_y'], axis=1)
    return test

In [0]:
batch_size = 10
wiki_pages_chunks = np.array_split(wiki_pages, batch_size)
i = 0
for chunks in wiki_pages_chunks:
    df = pd.DataFrame()
    for f in chunks:
        d = pd.read_json(f, orient='records', lines=True)
        df = df.append(d, ignore_index=True)
        print(f'read {f}')
    del d
    # train = merge_dfs(train['id'], df, train, i)
    df['match_id'] = df['id'].apply(lambda x: unidecode.unidecode(x))
    test = merge_dfs(test['match_id'], df, test, i)
    
    print(f'done chunk {i}')
    i += 1
    # if i ==2:
      # break
    

read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-001.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-002.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-003.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-004.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-005.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-006.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-007.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-008.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-009.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-010.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-011.jsonl
done chunk 0
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-012.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-013.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-014.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pag

In [0]:
test.drop(labels=['text'], axis=1, inplace=True)

In [0]:
test

,claim,claim_id,match_id,id,lines
0,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-disambiguation-RRB-,Henry_Spencer_-LRB-disambiguation-RRB-,0\tHenry Spencer is a Canadian computer progra...
1,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-Psych-RRB-,Henry_Spencer_-LRB-Psych-RRB-,"0\tHenry William Spencer , Jr. is a fictional ..."
2,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer,Henry_Spencer,0\tHenry Spencer -LRB- born 1955 -RRB- is a Ca...
3,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-fl._1402-RRB-,Henry_Spencer_-LRB-fl._1402-RRB-,0\tHenry Spencer -LRB- fl .\n1\t1402 -RRB- was...
4,Henry Spencer is played by a Greek actor.,89296,Lowndes_Henry_Davis,Lowndes_Henry_Davis,"0\tLowndes Henry Davis -LRB- December 13 , 183..."
...,...,...,...,...,...
174934,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-disambiguation-RRB-,The_Handmaid's_Tale_-LRB-disambiguation-RRB-,0\tThe Handmaid 's Tale is a 1985 novel by Mar...
174935,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale,The_Handmaid's_Tale,0\tThe Handmaid 's Tale is a 1985 dystopian no...
174936,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-film-RRB-,The_Handmaid's_Tale_-LRB-film-RRB-,0\tThe Handmaid 's Tale is a 1990 film adaptat...
174937,The Handmaid's Tale was approved for a second ...,51709,The_Handmaid's_Tale_-LRB-TV_series-RRB-,The_Handmaid's_Tale_-LRB-TV_series-RRB-,0\tThe Handmaid 's Tale is an American televis...


In [0]:
path = "/content/drive/My Drive/NLPWikiData/raw_test_data3.json"
test.to_json(path, orient='index')

In [0]:
raw_test_data = pd.read_json(path,orient='index')

In [0]:
raw_test_data.head(2)

,claim,claim_id,match_id,id,lines
0,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-disambiguation-RRB-,Henry_Spencer_-LRB-disambiguation-RRB-,0\tHenry Spencer is a Canadian computer progra...
1,Henry Spencer is played by a Greek actor.,89296,Henry_Spencer_-LRB-Psych-RRB-,Henry_Spencer_-LRB-Psych-RRB-,"0\tHenry William Spencer , Jr. is a fictional ..."


In [0]:
from tqdm import tqdm

test_processed_data = []
for t_data in tqdm(raw_test_data.itertuples()):
  # print(data)
  try:
    lines_with_slash_n = '\n'+t_data[5]
  except:
    print("Data:  ", t_data)
    continue

  lines = rule_split(lines_with_slash_n)
  lines = [line.strip().replace('\n', '') for line in lines]
  
  if len(lines) == 1 and lines[0] == '':
      lines = ['0']
  all_sentences = lines_to_items(lines)

  for i,sentence in enumerate(all_sentences):
    # temp_dict['doc_name'] = data[1]
    if len(sentence['sentence']) <3:
      continue
    temp_dict = dict()
    temp_dict['claim_id'] = t_data[2]
    temp_dict['claim'] = t_data[1]
    temp_dict['sentence'] = sentence['sentence']
    # temp_dict['label'] = t_data[4]
    # temp_dict['sent_label'] = sentence['line_num'] == data[1]
    temp_dict['sent_label'] = False
    # only for dev data
    temp_dict['sentno'] = sentence['line_num']
    temp_dict['doc_id'] = t_data[4]

    test_processed_data.append(temp_dict)


174939it [00:09, 17945.50it/s]


In [0]:
test_processed_data

[{'claim': 'Henry Spencer is played by a Greek actor.',
  'claim_id': 89296,
  'doc_id': 'Henry_Spencer_-LRB-disambiguation-RRB-',
  'sent_label': False,
  'sentence': 'Henry Spencer is a Canadian computer programmer and space enthusiast .',
  'sentno': 0},
 {'claim': 'Henry Spencer is played by a Greek actor.',
  'claim_id': 89296,
  'doc_id': 'Henry_Spencer_-LRB-disambiguation-RRB-',
  'sent_label': False,
  'sentence': 'Henry Spencer may also refer to :',
  'sentno': 3},
 {'claim': 'Henry Spencer is played by a Greek actor.',
  'claim_id': 89296,
  'doc_id': 'Henry_Spencer_-LRB-Psych-RRB-',
  'sent_label': False,
  'sentence': 'Henry William Spencer , Jr. is a fictional character on the American television dramedy Psych .',
  'sentno': 0},
 {'claim': 'Henry Spencer is played by a Greek actor.',
  'claim_id': 89296,
  'doc_id': 'Henry_Spencer_-LRB-Psych-RRB-',
  'sent_label': False,
  'sentence': 'He is played by American actor Corbin Bernsen .',
  'sentno': 1},
 {'claim': 'Henry Spe

In [0]:
test_new_df = pd.DataFrame(test_processed_data)
# only for dev data
test_new_df['org_sentence'] = test_new_df['sentence']

In [0]:
test_new_df

,claim_id,claim,sentence,sent_label,sentno,doc_id,org_sentence
0,89296,Henry Spencer is played by a Greek actor.,Henry Spencer is a Canadian computer programme...,False,0,Henry_Spencer_-LRB-disambiguation-RRB-,Henry Spencer is a Canadian computer programme...
1,89296,Henry Spencer is played by a Greek actor.,Henry Spencer may also refer to :,False,3,Henry_Spencer_-LRB-disambiguation-RRB-,Henry Spencer may also refer to :
2,89296,Henry Spencer is played by a Greek actor.,"Henry William Spencer , Jr. is a fictional cha...",False,0,Henry_Spencer_-LRB-Psych-RRB-,"Henry William Spencer , Jr. is a fictional cha..."
3,89296,Henry Spencer is played by a Greek actor.,He is played by American actor Corbin Bernsen .,False,1,Henry_Spencer_-LRB-Psych-RRB-,He is played by American actor Corbin Bernsen .
4,89296,Henry Spencer is played by a Greek actor.,Henry Spencer -LRB- born 1955 -RRB- is a Canad...,False,0,Henry_Spencer,Henry Spencer -LRB- born 1955 -RRB- is a Canad...
...,...,...,...,...,...,...,...
980362,51709,The Handmaid's Tale was approved for a second ...,The Handmaid 's Tale is an American television...,False,0,The_Handmaid's_Tale_-LRB-TV_series-RRB-,The Handmaid 's Tale is an American television...
980363,51709,The Handmaid's Tale was approved for a second ...,It was ordered by streaming service Hulu with ...,False,1,The_Handmaid's_Tale_-LRB-TV_series-RRB-,It was ordered by streaming service Hulu with ...
980364,51709,The Handmaid's Tale was approved for a second ...,The first three episodes of the series premier...,False,4,The_Handmaid's_Tale_-LRB-TV_series-RRB-,The first three episodes of the series premier...
980365,51709,The Handmaid's Tale was approved for a second ...,"In May 2017 , The Handmaid 's Tale was renewed...",False,5,The_Handmaid's_Tale_-LRB-TV_series-RRB-,"In May 2017 , The Handmaid 's Tale was renewed..."


In [0]:
# only for test data
seperator = "{#--#}"
test_new_df['docid_claimid_sentno'] = test_new_df.doc_id + seperator + test_new_df.claim_id.astype(str) + seperator + test_new_df.sentno.astype(str)
test_new_df.drop(labels=['doc_id', 'sentno'], axis=1, inplace=True)
test_new_df = test_new_df.drop_duplicates().reset_index(drop=True)

In [0]:
test_new_df.to_csv("/content/drive/My Drive/NLPWikiData/processed_test_data3.csv",index=False)

In [0]:
test_done_data = pd.read_csv("/content/drive/My Drive/NLPWikiData/processed_test_data3.csv")

In [0]:
test_done_data

,claim_id,claim,sentence,sent_label,org_sentence,docid_claimid_sentno
0,89296,Henry Spencer is played by a Greek actor.,Henry Spencer is a Canadian computer programme...,False,Henry Spencer is a Canadian computer programme...,Henry_Spencer_-LRB-disambiguation-RRB-{#--#}89...
1,89296,Henry Spencer is played by a Greek actor.,Henry Spencer may also refer to :,False,Henry Spencer may also refer to :,Henry_Spencer_-LRB-disambiguation-RRB-{#--#}89...
2,89296,Henry Spencer is played by a Greek actor.,"Henry William Spencer , Jr. is a fictional cha...",False,"Henry William Spencer , Jr. is a fictional cha...",Henry_Spencer_-LRB-Psych-RRB-{#--#}89296{#--#}0
3,89296,Henry Spencer is played by a Greek actor.,He is played by American actor Corbin Bernsen .,False,He is played by American actor Corbin Bernsen .,Henry_Spencer_-LRB-Psych-RRB-{#--#}89296{#--#}1
4,89296,Henry Spencer is played by a Greek actor.,Henry Spencer -LRB- born 1955 -RRB- is a Canad...,False,Henry Spencer -LRB- born 1955 -RRB- is a Canad...,Henry_Spencer{#--#}89296{#--#}0
...,...,...,...,...,...,...
980286,51709,The Handmaid's Tale was approved for a second ...,The Handmaid 's Tale is an American television...,False,The Handmaid 's Tale is an American television...,The_Handmaid's_Tale_-LRB-TV_series-RRB-{#--#}5...
980287,51709,The Handmaid's Tale was approved for a second ...,It was ordered by streaming service Hulu with ...,False,It was ordered by streaming service Hulu with ...,The_Handmaid's_Tale_-LRB-TV_series-RRB-{#--#}5...
980288,51709,The Handmaid's Tale was approved for a second ...,The first three episodes of the series premier...,False,The first three episodes of the series premier...,The_Handmaid's_Tale_-LRB-TV_series-RRB-{#--#}5...
980289,51709,The Handmaid's Tale was approved for a second ...,"In May 2017 , The Handmaid 's Tale was renewed...",False,"In May 2017 , The Handmaid 's Tale was renewed...",The_Handmaid's_Tale_-LRB-TV_series-RRB-{#--#}5...


# **Pre-Process - Dev Data**

In [0]:
wiki_pages = glob.glob('/content/drive/My Drive/NLPWikiData/wiki-pages/*')

In [0]:
df = pd.DataFrame()

In [0]:
dev_json_path = "/content/drive/My Drive/NLPWikiData/doc_ret_dev3.jsonl"
devdf = pd.read_json(dev_json_path, lines=True, orient='records')
# devdf = devdf[devdf['verifiable'] == 'VERIFIABLE']

In [0]:
devdf

,id,verifiable,claim,evidence,pred_evidence
0,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"[[[108548, None, None, None]]]","[Colin_Kaepernick, Maio_Regional_Football_Asso..."
1,194462,NOT VERIFIABLE,Tilda Swinton is a vegan.,"[[[227768, None, None, None]]]","[Tilda_Swinton, List_of_awards_and_nominations..."
2,137334,VERIFIABLE,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0...","[Soul_Food_-LRB-Oblivians_album-RRB-, Soul_Foo..."
3,166626,NOT VERIFIABLE,Anne Rice was born in New Jersey.,"[[[191656, None, None, None], [191657, None, N...","[Anne_Rice, New_Jersey, New_Jersey_-LRB-disamb..."
4,111897,VERIFIABLE,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1...","[Telemundo, Original_English-language_manga, L..."
...,...,...,...,...,...
19993,8538,VERIFIABLE,Hermit crabs are arachnids.,"[[[15450, 19262, Hermit_crab, 0], [15450, 1926...","[The_Mad_Hermit, Matthew_the_Hermit, Felix_the..."
19994,145641,VERIFIABLE,Michael Hutchence died on a boat.,"[[[168967, 182663, Michael_Hutchence, 15]]]","[Michael_Hutchence, Michael_Hutchence_-LRB-alb..."
19995,87517,VERIFIABLE,The Cyclades are located to the southeast of G...,"[[[104709, 118125, Cyclades, 0]]]","[Greece_-LRB-disambiguation-RRB-, Greece_-LRB-..."
19996,111816,NOT VERIFIABLE,Theresa May worked the docks.,"[[[131223, None, None, None]]]","[Theresa_May_-LRB-disambiguation-RRB-, Theresa..."


In [0]:
values = devdf['pred_evidence'].explode()
index = values.index

In [0]:
values

0                                    Colin_Kaepernick
0                  Maio_Regional_Football_Association
0                            Louisiana_Football_Field
0                            Mackay_Regional_Football
0        Maldivian_Third_Division_Football_Tournament
                             ...                     
19997        Trouble_-LRB-Cage_the_Elephant_song-RRB-
19997                  Trouble_-LRB-Sailor_album-RRB-
19997                          Trouble_-LRB-film-RRB-
19997            Trouble_-LRB-Bonnie_McKee_album-RRB-
19997                      Trouble_-LRB-magazine-RRB-
Name: pred_evidence, Length: 175658, dtype: object

In [0]:
index

Int64Index([    0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,
            ...
            19997, 19997, 19997, 19997, 19997, 19997, 19997, 19997, 19997,
            19997],
           dtype='int64', length=175658)

In [0]:
dev = pd.DataFrame(list(values))

In [0]:
dev

,0
0,Colin_Kaepernick
1,Maio_Regional_Football_Association
2,Louisiana_Football_Field
3,Mackay_Regional_Football
4,Maldivian_Third_Division_Football_Tournament
...,...
175653,Trouble_-LRB-Cage_the_Elephant_song-RRB-
175654,Trouble_-LRB-Sailor_album-RRB-
175655,Trouble_-LRB-film-RRB-
175656,Trouble_-LRB-Bonnie_McKee_album-RRB-


In [0]:
# values2 = devdf['evidence'].explode().explode()
# index2 = values2.index

In [0]:
# dev = pd.DataFrame(list(values2))

In [0]:
# dev

In [0]:
dev['claim']=devdf['claim'][index].reset_index()['claim']

In [0]:
dev['claim_id']=devdf['id'][index].reset_index()['id']

In [0]:
dev

,0,claim,claim_id
0,Colin_Kaepernick,Colin Kaepernick became a starting quarterback...,91198
1,Maio_Regional_Football_Association,Colin Kaepernick became a starting quarterback...,91198
2,Louisiana_Football_Field,Colin Kaepernick became a starting quarterback...,91198
3,Mackay_Regional_Football,Colin Kaepernick became a starting quarterback...,91198
4,Maldivian_Third_Division_Football_Tournament,Colin Kaepernick became a starting quarterback...,91198
...,...,...,...
175653,Trouble_-LRB-Cage_the_Elephant_song-RRB-,Trouble with the Curve is a television show.,81957
175654,Trouble_-LRB-Sailor_album-RRB-,Trouble with the Curve is a television show.,81957
175655,Trouble_-LRB-film-RRB-,Trouble with the Curve is a television show.,81957
175656,Trouble_-LRB-Bonnie_McKee_album-RRB-,Trouble with the Curve is a television show.,81957


In [0]:
# dev['label'] = devdf['label'][index].reset_index()['label']

In [0]:
dev.columns = ['id', 'claim', 'claim_id']

In [0]:
dev

,id,claim,claim_id
0,Colin_Kaepernick,Colin Kaepernick became a starting quarterback...,91198
1,Maio_Regional_Football_Association,Colin Kaepernick became a starting quarterback...,91198
2,Louisiana_Football_Field,Colin Kaepernick became a starting quarterback...,91198
3,Mackay_Regional_Football,Colin Kaepernick became a starting quarterback...,91198
4,Maldivian_Third_Division_Football_Tournament,Colin Kaepernick became a starting quarterback...,91198
...,...,...,...
175653,Trouble_-LRB-Cage_the_Elephant_song-RRB-,Trouble with the Curve is a television show.,81957
175654,Trouble_-LRB-Sailor_album-RRB-,Trouble with the Curve is a television show.,81957
175655,Trouble_-LRB-film-RRB-,Trouble with the Curve is a television show.,81957
175656,Trouble_-LRB-Bonnie_McKee_album-RRB-,Trouble with the Curve is a television show.,81957


In [0]:
dev['match_id'] = dev['id'].apply(lambda x: unidecode.unidecode(x))

In [0]:
# def split_into_id_and_lines(s):
#     lines = s.split('\n')
#     result = []
#     for line in lines:
#         sentence_id, text = line.split('\t', 1)
#         result.append([sentence_id, text])
#     return result

def merge_dfs(pages, wiki_dump, dev, i):
    # tmp = wiki_dump[wiki_dump['id'].isin(pages)]
    # # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    # train = train.merge(tmp, how='left', left_on='id', right_on='id')
    # if i > 0:
    #     train['text'] = train['text_x'].fillna(train['text_y'])
    #     train['lines'] = train['lines_x'].fillna(train['lines_y'])
    #     train = train.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y'], axis=1)
    # return train


    tmp = wiki_dump[wiki_dump['match_id'].isin(pages)]
    # tmp['lines'] = tmp['lines'].apply(lambda x: split_into_id_and_lines(x))
    dev = dev.merge(tmp, how='left', left_on='match_id', right_on='match_id')
    if i > 0:
        dev['id'] = dev['id_x'].fillna(dev['id_y'])
        dev['text'] = dev['text_x'].fillna(dev['text_y'])
        dev['lines'] = dev['lines_x'].fillna(dev['lines_y'])
        dev = dev.drop(labels=['text_x', 'text_y', 'lines_x', 'lines_y', 'id_x', 'id_y'], axis=1)
    return dev

In [0]:
batch_size = 10
wiki_pages_chunks = np.array_split(wiki_pages, batch_size)
i = 0
for chunks in wiki_pages_chunks:
    df = pd.DataFrame()
    for f in chunks:
        d = pd.read_json(f, orient='records', lines=True)
        df = df.append(d, ignore_index=True)
        print(f'read {f}')
    del d
    # train = merge_dfs(train['id'], df, train, i)
    df['match_id'] = df['id'].apply(lambda x: unidecode.unidecode(x))
    dev = merge_dfs(dev['match_id'], df, dev, i)
    
    print(f'done chunk {i}')
    i += 1
    # if i ==2:
      # break
    

read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-001.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-002.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-003.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-004.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-005.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-006.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-007.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-008.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-009.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-010.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-011.jsonl
done chunk 0
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-012.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-013.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pages/wiki-014.jsonl
read /content/drive/My Drive/NLPWikiData/wiki-pag

In [0]:
dev.drop(labels=['text'], axis=1, inplace=True)

In [0]:
dev

,claim,claim_id,match_id,id,lines
0,Colin Kaepernick became a starting quarterback...,91198,Colin_Kaepernick,Colin_Kaepernick,0\tColin Rand Kaepernick -LRB- -LSB- ` kæpərnɪ...
1,Colin Kaepernick became a starting quarterback...,91198,Maio_Regional_Football_Association,Maio_Regional_Football_Association,0\tMaio Regional Football Association -LRB- Po...
2,Colin Kaepernick became a starting quarterback...,91198,Louisiana_Football_Field,Louisiana_Football_Field,0\tLouisiana Football Field is a football stad...
3,Colin Kaepernick became a starting quarterback...,91198,Mackay_Regional_Football,Mackay_Regional_Football,0\tMackay Regional Football is a Football Quee...
4,Colin Kaepernick became a starting quarterback...,91198,Maldivian_Third_Division_Football_Tournament,Maldivian_Third_Division_Football_Tournament,0\tThird Division Football Tournament is a foo...
...,...,...,...,...,...
175889,Trouble with the Curve is a television show.,81957,Trouble_-LRB-Cage_the_Elephant_song-RRB-,Trouble_-LRB-Cage_the_Elephant_song-RRB-,0\t`` Trouble '' is a song by American alterna...
175890,Trouble with the Curve is a television show.,81957,Trouble_-LRB-Sailor_album-RRB-,Trouble_-LRB-Sailor_album-RRB-,0\tTrouble is the second album by the British ...
175891,Trouble with the Curve is a television show.,81957,Trouble_-LRB-film-RRB-,Trouble_-LRB-film-RRB-,0\tTrouble is a 1933 British comedy film direc...
175892,Trouble with the Curve is a television show.,81957,Trouble_-LRB-Bonnie_McKee_album-RRB-,Trouble_-LRB-Bonnie_McKee_album-RRB-,0\tTrouble is the debut album of American sing...


In [0]:
dev_path = "/content/drive/My Drive/NLPWikiData/raw_dev_data3.json"
dev.to_json(dev_path, orient='index')

In [0]:
raw_dev_data = pd.read_json(dev_path,orient='index')

In [0]:
raw_dev_data.head(2)

,claim,claim_id,match_id,id,lines
0,Colin Kaepernick became a starting quarterback...,91198,Colin_Kaepernick,Colin_Kaepernick,0\tColin Rand Kaepernick -LRB- -LSB- ` kæpərnɪ...
1,Colin Kaepernick became a starting quarterback...,91198,Maio_Regional_Football_Association,Maio_Regional_Football_Association,0\tMaio Regional Football Association -LRB- Po...


In [0]:
from tqdm import tqdm

dev_processed_data = []
for d_data in tqdm(raw_dev_data.itertuples()):
  # print(data)
  try:
    lines_with_slash_n = '\n'+d_data[5]
  except:
    print("Data:  ", d_data)
    continue

  lines = rule_split(lines_with_slash_n)
  lines = [line.strip().replace('\n', '') for line in lines]
  
  if len(lines) == 1 and lines[0] == '':
      lines = ['0']
  all_sentences = lines_to_items(lines)

  for i,sentence in enumerate(all_sentences):
    # temp_dict['doc_name'] = data[1]
    if len(sentence['sentence']) <3:
      continue
    temp_dict = dict()
    temp_dict['claim_id'] = d_data[2]
    temp_dict['claim'] = d_data[1]
    temp_dict['sentence'] = sentence['sentence']
    # temp_dict['label'] = d_data[4]
    # temp_dict['sent_label'] = sentence['line_num'] == data[1]
    # temp_dict['sent_label'] = False
    # only for dev/test data
    temp_dict['sentno'] = sentence['line_num']
    temp_dict['doc_id'] = d_data[4]

    dev_processed_data.append(temp_dict)


175894it [00:09, 18181.28it/s]


In [0]:
dev_new_df = pd.DataFrame(dev_processed_data)
# only for dev data
dev_new_df['org_sentence'] = dev_new_df['sentence']

In [0]:
dev_new_df

,claim_id,claim,sentence,sentno,doc_id,org_sentence
0,91198,Colin Kaepernick became a starting quarterback...,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,0,Colin_Kaepernick,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...
1,91198,Colin Kaepernick became a starting quarterback...,Kaepernick played collegiate football at the U...,1,Colin_Kaepernick,Kaepernick played collegiate football at the U...
2,91198,Colin Kaepernick became a starting quarterback...,Kaepernick was selected by the San Francisco 4...,2,Colin_Kaepernick,Kaepernick was selected by the San Francisco 4...
3,91198,Colin Kaepernick became a starting quarterback...,Kaepernick began his professional career as a ...,5,Colin_Kaepernick,Kaepernick began his professional career as a ...
4,91198,Colin Kaepernick became a starting quarterback...,He remained the team 's starting quarterback f...,6,Colin_Kaepernick,He remained the team 's starting quarterback f...
...,...,...,...,...,...,...
992972,81957,Trouble with the Curve is a television show.,McKee had essentially completed the album in N...,5,Trouble_-LRB-Bonnie_McKee_album-RRB-,McKee had essentially completed the album in N...
992973,81957,Trouble with the Curve is a television show.,"Pleased with the more layered sound , she deci...",6,Trouble_-LRB-Bonnie_McKee_album-RRB-,"Pleased with the more layered sound , she deci..."
992974,81957,Trouble with the Curve is a television show.,This delayed the album 's release by a year .,7,Trouble_-LRB-Bonnie_McKee_album-RRB-,This delayed the album 's release by a year .
992975,81957,Trouble with the Curve is a television show.,Trouble is a free independent monthly magazine...,0,Trouble_-LRB-magazine-RRB-,Trouble is a free independent monthly magazine...


In [0]:
# only for test/dev data
seperator = "{#--#}"
dev_new_df['docid_claimid_sentno'] = dev_new_df.doc_id + seperator + dev_new_df.claim_id.astype(str) + seperator + dev_new_df.sentno.astype(str)
dev_new_df.drop(labels=['doc_id', 'sentno'], axis=1, inplace=True)
dev_new_df = dev_new_df.drop_duplicates().reset_index(drop=True)

In [0]:
dev_new_df

,claim_id,claim,sentence,org_sentence,docid_claimid_sentno
0,91198,Colin Kaepernick became a starting quarterback...,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,Colin_Kaepernick{#--#}91198{#--#}0
1,91198,Colin Kaepernick became a starting quarterback...,Kaepernick played collegiate football at the U...,Kaepernick played collegiate football at the U...,Colin_Kaepernick{#--#}91198{#--#}1
2,91198,Colin Kaepernick became a starting quarterback...,Kaepernick was selected by the San Francisco 4...,Kaepernick was selected by the San Francisco 4...,Colin_Kaepernick{#--#}91198{#--#}2
3,91198,Colin Kaepernick became a starting quarterback...,Kaepernick began his professional career as a ...,Kaepernick began his professional career as a ...,Colin_Kaepernick{#--#}91198{#--#}5
4,91198,Colin Kaepernick became a starting quarterback...,He remained the team 's starting quarterback f...,He remained the team 's starting quarterback f...,Colin_Kaepernick{#--#}91198{#--#}6
...,...,...,...,...,...
992972,81957,Trouble with the Curve is a television show.,McKee had essentially completed the album in N...,McKee had essentially completed the album in N...,Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992973,81957,Trouble with the Curve is a television show.,"Pleased with the more layered sound , she deci...","Pleased with the more layered sound , she deci...",Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992974,81957,Trouble with the Curve is a television show.,This delayed the album 's release by a year .,This delayed the album 's release by a year .,Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992975,81957,Trouble with the Curve is a television show.,Trouble is a free independent monthly magazine...,Trouble is a free independent monthly magazine...,Trouble_-LRB-magazine-RRB-{#--#}81957{#--#}0


In [0]:
dev_new_df.to_csv("/content/drive/My Drive/NLPWikiData/processed_dev_data3.csv",index=False)

In [0]:
dev_done_data = pd.read_csv("/content/drive/My Drive/NLPWikiData/processed_dev_data3.csv")

In [0]:
dev_done_data

,claim_id,claim,sentence,org_sentence,docid_claimid_sentno
0,91198,Colin Kaepernick became a starting quarterback...,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,Colin_Kaepernick{#--#}91198{#--#}0
1,91198,Colin Kaepernick became a starting quarterback...,Kaepernick played collegiate football at the U...,Kaepernick played collegiate football at the U...,Colin_Kaepernick{#--#}91198{#--#}1
2,91198,Colin Kaepernick became a starting quarterback...,Kaepernick was selected by the San Francisco 4...,Kaepernick was selected by the San Francisco 4...,Colin_Kaepernick{#--#}91198{#--#}2
3,91198,Colin Kaepernick became a starting quarterback...,Kaepernick began his professional career as a ...,Kaepernick began his professional career as a ...,Colin_Kaepernick{#--#}91198{#--#}5
4,91198,Colin Kaepernick became a starting quarterback...,He remained the team 's starting quarterback f...,He remained the team 's starting quarterback f...,Colin_Kaepernick{#--#}91198{#--#}6
...,...,...,...,...,...
992972,81957,Trouble with the Curve is a television show.,McKee had essentially completed the album in N...,McKee had essentially completed the album in N...,Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992973,81957,Trouble with the Curve is a television show.,"Pleased with the more layered sound , she deci...","Pleased with the more layered sound , she deci...",Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992974,81957,Trouble with the Curve is a television show.,This delayed the album 's release by a year .,This delayed the album 's release by a year .,Trouble_-LRB-Bonnie_McKee_album-RRB-{#--#}8195...
992975,81957,Trouble with the Curve is a television show.,Trouble is a free independent monthly magazine...,Trouble is a free independent monthly magazine...,Trouble_-LRB-magazine-RRB-{#--#}81957{#--#}0
